In [1]:
import numpy as np
import pandas as pd
import os
import glob
import csv
from pcapfile import savefile
from pcapfile.protocols.linklayer import ethernet
from pcapfile.protocols.network import ip
from pcapfile.protocols.transport import tcp
import binascii
from datetime import datetime
import pytz

ImportError: No module named 'pcapfile'

In [ ]:
csv_record_path = '/media/mo/HDD/intrusion_detection/dataset/AttacksRecords/dataset/'

In [ ]:
#Processed Traffic Data
files = ['Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv',
        'Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv',
        'Friday-16-02-2018_TrafficForML_CICFlowMeter.csv',
        'Thuesday-20-02-2018_TrafficForML_CICFlowMeter.csv',
        'Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv',
        'Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv',
        'Friday-23-02-2018_TrafficForML_CICFlowMeter.csv',
        'Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv',
        'Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv',
        'Friday-02-03-2018_TrafficForML_CICFlowMeter.csv']

In [ ]:
def read_file (file_):
    #input the name for the CSV file
    with open(file_, 'r') as f:
        contents = [x.split(',') for x in f.readlines()]
        f.flush()
    return contents

def day_attack(file_):
    #input the content of csv file as np array
    return [item for item in np.unique(file_[:,-1]) if item!='Benign\n' and item!='Label\n']

def file_features(file_):
    #input the content of csv file as np array
    return file_[0]

def extract_flowdata(file_, attack_names):
    #input the content of csv file as np array
    dstPort=[]
    protocol=[]
    #attack_records = [file_[0].tolist()]
    attack_records = [file_[0]]
    for name in attack_names:
        #record=[item.tolist() for item in file_ if item[-1]==name]
        record=[item for item in file_ if item[-1]==name]
        attack_records.extend(record)
        dstPort.append(np.unique(np.array(record)[:,0]).tolist())
        protocol.append(np.unique(np.array(record)[:,1]).tolist())
    return np.array(attack_records), dstPort, protocol

def write_to_file(file_, name):
    #input attack records
    df = pd.DataFrame(data=file_[1:], columns=file_[0])
    df.to_csv(name, index=False)
    return df
        

In [ ]:
def normalized_timestamp(ts):
    return(str(datetime.utcfromtimestamp(ts)))


def convert_datetime_timezone(dt, tz1, tz2):
    tz1 = pytz.timezone(tz1)
    tz2 = pytz.timezone(tz2)
    dt = datetime.strptime(dt,"%Y-%m-%d %H:%M:%S")
    dt = tz1.localize(dt)
    dt = dt.astimezone(tz2)
    dt = dt.strftime("%Y-%m-%d %H:%M:%S")
    return dt

def get_all_flows(capdata):
    #input capdata
    flows =[]
    timestamps=[]
    for pkt in capdata.packets:
        eth_frame = ethernet.Ethernet(pkt.raw())
        try:
            ip_packet = ip.IP(binascii.unhexlify(eth_frame.payload))
            tcp_packet = tcp.TCP(binascii.unhexlify(eth_frame.payload))
        except:
            continue
        flows.append([ip_packet.src.decode("utf-8"), ip_packet.dst.decode("utf-8"), str(tcp_packet.src_port), str(tcp_packet.dst_port), str(ip_packet.p)])
        timestamps.append(normalized_timestamp(pkt.timestamp))
    return np.unique(flows, axis=0), flows,timestamps


def get_attack_flows(flows, attacker_ip):
    #input unique_flows
    if attacker_ip in np.unique(flows[:,0]):
        attacker_fwd_flows=[item.tolist() for item in flows if item[0]==attacker_ip]
    else:
        attacker_fwd_flows=[]
        print('Attacker IP does not exist !!!')
        
    if attacker_ip in np.unique(flows[:,1]):
        attacker_bwd_flows=[item.tolist() for item in flows if item[1]==attacker_ip]
    else:
        attacker_bwd_flows=[]
        print('Attacker IP does not exist !!!')
    
    return attacker_fwd_flows, attacker_bwd_flows   


def write_labels(attacker_fwd_flows, attacker_bwd_flows, attack):
    uni_labeled_file = open(csv_dist_path+"uni_labels.txt", 'a')
    for item in attacker_fwd_flows:
        for initem in item:
            uni_labeled_file.write(initem +',')
        uni_labeled_file.write(attack)

    bi_labeled_file = open(csv_dist_path+"bi_labels.txt", 'a')
    for item in attacker_fwd_flows:
        for initem in item:
            bi_labeled_file.write(initem +',')
        bi_labeled_file.write(attack)
    for item in attacker_bwd_flows:
        for initem in item:
            bi_labeled_file.write(initem +',')
        bi_labeled_file.write(attack)
    uni_labeled_file.close()
    bi_labeled_file.close()
    
def attack_active_time(attacker_fwd_flows,flows,timestamps):
    for item in attacker_fwd_flows:
        i = []
        j = 0
        for idx, itemx in enumerate(flows):
            if item==itemx:
                i.append(idx)
                j+=1

        begin_time=convert_datetime_timezone(timestamps[i[0]], "UTC", "Canada/Atlantic")
        end_time=convert_datetime_timezone(timestamps[i[-1]], "UTC", "Canada/Atlantic")
        print("Flow: {}\n\nBegin: {}   End: {}\n\nNumber of packets: {}\n" .format(item, begin_time,end_time,j))

        
def get_all_flows_2(cap, attackers, write_to_file):
    #input file object and a file to write the flows in formation 
    flow_file = open (write_to_file,'a') 
    flows =[]
    timestamps=[]
    for pkt in savefile.load_savefile(cap, lazy=True).packets:
        eth_frame = ethernet.Ethernet(pkt.raw())
        try:
            ip_packet = ip.IP(binascii.unhexlify(eth_frame.payload))
            tcp_packet = tcp.TCP(binascii.unhexlify(eth_frame.payload))
        except:
            continue
        if ip_packet.src.decode("utf-8") in attackers:
            flow_file.write('{},{},{},{},{},{}\n'.format(ip_packet.src.decode("utf-8"), ip_packet.dst.decode("utf-8"), str(tcp_packet.src_port), str(tcp_packet.dst_port), str(ip_packet.p),normalized_timestamp(pkt.timestamp)))
        else:
            continue
    flow_file.close()
    
def read_unique_flows(file_):
    #file_ =csv_dist_path+file_
    with open (file_,'r') as f:
        contents = f.readlines()
        f.flush()
    return list(set(contents))



def get_attacks_labels(unique_flows, attack_time, attack_name,labels_file): 
    flows = [item.split(',') for item in unique_flows]
    #time = [item.split()[-1] for item in np.array(flows)[:,-1]]
    attack_flow = open(labels_file,'a')
    for item in flows:
        time = item[-1].split()[-1]
        if time > attack_time[0] and time < attack_time[1]:
            attack_flow.write('{},{},{},{},{},{}\n'.format(item[0],item[1],item[2],item[3],item[4],attack_name))
    attack_flow.close()   

In [ ]:
def order_flowid(id_):
    flowid = id_.split('-')
    return '-'.join([flowid[1],flowid[0],flowid[3],flowid[2],flowid[4]]) 

def clear_labels(data):
    for clx, item in enumerate (data):
        if clx!=0:
            data[clx][0] = '-'.join([item[1],item[3],item[2],item[4],item[5]])
        if clx!=0 and item[-1] == 'No Label\n':
            data[clx][-1]='Benign\n'
        else:
            continue
    return data

def label_flows (data, attack_source, attack_time, attack_names, save_path, infiltration=False):
    data = np.array(data)
    #not_flipped=True
    for ttx, attack_name in enumerate(attack_names):
        for source in attack_source[ttx]:
            for idx, record in enumerate(data):
                if idx == 0:
                    continue
                else:
                    #if ttx == 0 and not_flipped:
                    #    data[idx][0]= order_flowid(record[0])
                    if infiltration:
                        if record[3] == source and record[6]>=attack_time[ttx][0] and record[6]<=attack_time[ttx][1]:
                            data[idx][-1]= attack_name+'\n'
                        else:
                            continue
                    else:
                        if record[1] == source and record[6]>=attack_time[ttx][0] and record[6]<=attack_time[ttx][1]:
                            data[idx][-1]= attack_name+'\n'
                        else:
                            continue
            #not_flipped=False
    
    data = clear_labels(data)
    if save_path != False:
        np.savetxt(save_path, data, delimiter=",", fmt='%s')
    return data         


# Day 1: FTP-BruteForce AND SSH-Bruteforce


In [ ]:
#read processed data
data_cicflow = read_file(csv_record_path +'Wednesday-14-02-2018/UCAP172.31.69.25_Flow.csv')
features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n".format(features_cicflow))

In [ ]:
len(data_cicflow)

In [ ]:
attack_name = ['FTP-BruteForce','SSH-Bruteforce']
attack_source = [['18.221.219.4'], ['13.58.98.64']]
attack_time = [['14/02/2018 09:32 AM','14/02/2018 11:09 AM'], 
               ['14/02/2018 01:01 PM','14/02/2018 02:31 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name,save_path=False)  
data.shape

In [ ]:
data[-1]

In [ ]:
labels = set(data[:,-1])
labels

In [ ]:
records = write_to_file(data, csv_record_path+files[0])
records.head()

# Day 2: DoS-GoldenEye AND DoS-Slowloris


In [ ]:
#read processed data
data_cicflow = read_file(csv_record_path +'Thursday-15-02-2018/UCAP172.31.69.25_Flow.csv')
features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(features_cicflow, len(data_cicflow)))

In [ ]:
attack_name = ['DoS-GoldenEye','DoS-Slowloris']
attack_source = [['18.219.211.138'], ['18.217.165.70']]
attack_time = [['15/02/2018 08:26 AM','15/02/2018 09:09 AM'], 
               ['15/02/2018 09:59 AM','15/02/2018 10:40 AM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name,save_path=False)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[1])
records.head()

# Day 3: DoS-SlowHTTPTest AND DoS-Hulk


In [ ]:
data_cicflow = read_file(csv_record_path +'Friday-16-02-2018/UCAP172.31.69.25-part1.pcap_Flow.csv')
data_cicflow += read_file(csv_record_path +'Friday-16-02-2018/UCAP172.31.69.25-part2.pcap_Flow.csv')[1:]

#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['DoS-SlowHTTPTest','DoS-Hulk']
attack_source = [['13.59.126.31'], ['18.219.193.20']]
attack_time = [['16/02/2018 09:12 AM','16/02/2018 10:08 AM'], 
               ['16/02/2018 12:45 PM','16/02/2018 13:19 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=csv_record_path+files[2])  
data.shape

#  Day 4: DDoS attacks-LOIC-HTTP AND DDoS-LOIC-UDP



In [ ]:
data_cicflow = read_file(csv_record_path +'Thuesday-20-02-2018/UCAP172.31.69.25_Flow.csv')
#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['DDoS attacks-LOIC-HTTP','DDoS-LOIC-UDP']
attack_source = [['18.218.115.60',
                    '18.219.9.1',
                    '18.219.32.43',
                    '18.218.55.126',
                    '52.14.136.135',
                    '18.219.5.43',
                    '18.216.200.189',
                    '18.218.229.235',
                    '18.218.11.51',
                    '18.216.24.42'], 
                 ['18.218.115.60',
                    '18.219.9.1',
                    '18.219.32.43',
                    '18.218.55.126',
                    '52.14.136.135',
                    '18.219.5.43',
                    '18.216.200.189',
                    '18.218.229.235',
                    '18.218.11.51',
                    '18.216.24.42']]

attack_time = [['20/02/2018 09:12 AM','20/02/2018 10:17 AM'], 
               ['20/02/2018 12:13 PM','20/02/2018 12:32 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[3])
records.head()

#  Day 5: DDOS attack-HOIC AND DDoS-LOIC-UDP


In [ ]:
data_cicflow = read_file(csv_record_path +'Wednesday-21-02-2018/UCAP172.31.69.28 part 1_Flow.csv')
data_cicflow+= read_file(csv_record_path +'Wednesday-21-02-2018/UCAP172.31.69.28 part 2_Flow.csv')[1:]

#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['DDOS-LOIC-UDP','DDOS-HOIC']
attack_source = [['18.218.115.60',
                    '18.219.9.1',
                    '18.219.32.43',
                    '18.218.55.126',
                    '52.14.136.135',
                    '18.219.5.43',
                    '18.216.200.189',
                    '18.218.229.235',
                    '18.218.11.51',
                    '18.216.24.42'], 
                 ['18.218.115.60',
                    '18.219.9.1',
                    '18.219.32.43',
                    '18.218.55.126',
                    '52.14.136.135',
                    '18.219.5.43',
                    '18.216.200.189',
                    '18.218.229.235',
                    '18.218.11.51',
                    '18.216.24.42']]

attack_time = [['21/02/2018 09:09 AM','21/02/2018 09:43 AM'], 
               ['21/02/2018 01:05 PM','21/02/2018 02:05 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[4])
records.head()

#  Day 6: Brute Force -Web AND Brute Force -XSS AND SQL Injection


# Brute Force -Web

In [ ]:
data_cicflow = read_file(csv_record_path +'Thursday-22-02-2018/UCAP172.31.69.28_Flow.csv')

#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['Brute Force-Web','Brute Force-XSS','SQL Injection']
attack_source = [['18.218.115.60'], 
                 ['18.218.115.60'],
                 ['18.218.115.60']]

attack_time = [['22/02/2018 09:17 AM','22/02/2018 10:24 AM'], 
               ['22/02/2018 01:00 PM','22/02/2018 01:29 PM'],
               ['22/02/2018 03:15 PM','22/02/2018 03:29 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[5])
records.head()

#  Day 7: Brute Force -Web AND Brute Force -XSS AND SQL Injection


In [ ]:
data_cicflow = read_file(csv_record_path +'Friday-23-02-2018/UCAP172.31.69.28_Flow.csv')

#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['Brute Force-Web','Brute Force-XSS','SQL Injection']
attack_source = [['18.218.115.60'], 
                 ['18.218.115.60'],
                 ['18.218.115.60']]

attack_time = [['23/02/2018 09:03 AM','23/02/2018 10:03 AM'], 
               ['23/02/2018 01:00 PM','23/02/2018 02:10 PM'],
               ['23/02/2018 02:05 PM','23/02/2018 02:18 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[6])
records.head()

#  Day 8: Infiltration


In [ ]:
data_cicflow = read_file(csv_record_path +'Wednesday-28-02-2018/capEC2AMAZ-O4EL3NG-172.31.69.24- part1_Flow.csv')
data_cicflow+= read_file(csv_record_path +'Wednesday-28-02-2018/capEC2AMAZ-O4EL3NG-172.31.69.24-part2_Flow.csv')[1:]

#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['Infiltration','Infiltration']
attack_source = [['13.58.225.34'], 
                 ['13.58.225.34']]

attack_time = [['28/02/2018 09:50 AM','28/02/2018 11:05 AM'], 
               ['28/02/2018 12:42 PM','28/02/2018 13:40 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False, infiltration=True)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[7])
records.head()

#  Day 9: Infiltration


In [ ]:
data_cicflow = read_file(csv_record_path +'Thursday-01-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.13 part1_Flow.csv')
data_cicflow+= read_file(csv_record_path +'Thursday-01-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.13 part2_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Thursday-01-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.13 part3_Flow.csv')[1:]

#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['Infiltration','Infiltration']
attack_source = [['13.58.225.34'], 
                 ['13.58.225.34']]

attack_time = [['01/03/2018 08:57 AM','01/03/2018 09:55 AM'], 
               ['01/03/2018 01:00 PM','01/03/2018 02:37 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False, infiltration=True)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[8])
records.head()

#  Day 10: Bot

In [ ]:
data_cicflow = read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.6_Flow.csv')
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.8_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.10_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.12_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.14_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.17_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.23_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.26_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.29_Flow.csv')[1:]
data_cicflow+= read_file(csv_record_path +'Friday-02-03-2018/capEC2AMAZ-O4EL3NG-172.31.69.30_Flow.csv')[1:]


#features_cicflow = file_features(data_cicflow)
print("Features:\n {}\n Data: {}\n".format(data_cicflow[0], len(data_cicflow)))

In [ ]:
attack_name = ['Bot']
attack_source = [['18.219.211.138']]

attack_time = [['02/03/2018 09:11 AM','02/03/2018 10:34 AM'], 
               ['02/03/2018 01:24 PM','02/03/2018 02:55 PM']]

data = label_flows(data_cicflow, attack_source, attack_time, attack_name, save_path=False)  
data.shape

In [ ]:
set(data[:,-1])

In [ ]:
records = write_to_file(data, csv_record_path+files[9])
records.head()